In [ ]:
!pip install google-generativeai langchain langgraph httpx langchain-google-genai

In [ ]:
from typing import Dict, TypedDict, Optional
from langgraph.graph import StateGraph, END
import random
import time

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
import os
# 1f. Import Google Colab library to access key
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY)

def llm(x):
    return model.invoke(x).content

In [ ]:
class GraphState(TypedDict):
  feedback:Optional[str]
  history: Optional[str]
  code: Optional[str]
  specialization: Optional[str]
  rating: Optional[str]
  iterations: Optional[int]
  code_compare: Optional[str]
  actual_code: Optional[str]
  is_executable: Optional[str]

workflow = StateGraph(GraphState)

In [ ]:
reviewer_start= "You are Code reviewer specialized in {}.\
You need to review the given code following PEP8 guidelines and potential bugs\
and point out issues as bullet list.\
Code:\n {}"

coder_start = "You are a Coder specialized in {}.\
Improve the given code given the following guidelines. Guideline:\n {} \n \
Code:\n {} \n \
Output just the improved code and nothing else."

rating_start = "Rate the skills of the coder on a scale of 10 given the Code review cycle with a short reason.\
Code review:\n {} \n "

code_comparison = "Compare the two code snippets and rate on a scale of 10 to both. Dont output the codes.Revised Code: \n {} \n Actual Code: \n {}"

classify_feedback = "Are all feedback mentioned resolved in the code? Output just Yes or No.\
Code: \n {} \n Feedback: \n {} \n"

def handle_coder(state):
    history = state.get('history', '').strip()
    feedback = state.get('feedback', '').strip()
    code =  state.get('code','').strip()
    specialization = state.get('specialization','').strip()

    print("CODER rewriting...")

    code = llm(coder_start.format(specialization,feedback,code))
    return {'history':history+'\n CODER:\n'+code,'code':code}


def handle_reviewer(state):
    history = state.get('history', '').strip()
    code = state.get('code', '').strip()
    specialization = state.get('specialization','').strip()
    iterations = state.get('iterations')

    print("Reviewer working...")

    feedback = llm(reviewer_start.format(specialization,code))

    return {'history':history+"\n REVIEWER:\n"+feedback,'feedback':feedback,'iterations':iterations+1}


def handle_result(state):
    print("Review done...")

    history = state.get('history', '').strip()
    code1 = state.get('code', '').strip()
    code2 = state.get('actual_code', '').strip()
    rating  = llm(rating_start.format(history))

    code_compare = llm(code_comparison.format(code1,code2))
    return {'rating':rating,'code_compare':code_compare}

In [ ]:
workflow.add_node("handle_reviewer",handle_reviewer)
workflow.add_node("handle_coder",handle_coder)
workflow.add_node("handle_result",handle_result)

In [ ]:
def deployment_ready(state):
    deployment_ready = 1 if 'yes' in llm(classify_feedback.format(state.get('code'),state.get('feedback'))) else 0
    total_iterations = 1 if state.get('iterations')>5 else 0
    return "handle_result" if  deployment_ready or total_iterations else "handle_coder"

In [ ]:
workflow.add_conditional_edges(
    "handle_reviewer",
    deployment_ready,
    {
        "handle_result": "handle_result",
        "handle_coder": "handle_coder"
    }
)

workflow.set_entry_point("handle_reviewer")
workflow.add_edge('handle_coder', "handle_reviewer")
workflow.add_edge('handle_result', END)

In [ ]:
specialization = 'python'
problem = 'Generate code to train a Regression ML model using a tabular dataset following required preprocessing steps.'
code = llm(problem)

app = workflow.compile()
conversation = app.invoke({"history":code,"code":code,'actual_code':code,"specialization":specialization,'iterations':0},{"recursion_limit":100})

Reviewer working...
CODER rewriting...
Reviewer working...
CODER rewriting...
Reviewer working...
CODER rewriting...
Reviewer working...
CODER rewriting...
Reviewer working...
CODER rewriting...
Reviewer working...
Review done...


In [ ]:
print(conversation['code'])

```python
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Load the tabular dataset
try:
    data = pd.read_csv('data.csv')
except FileNotFoundError:
    print("The file 'data.csv' was not found.")
    exit()

# Check if the target_column column exists in the data
if 'target_column' not in data.columns:
    print("The 'target_column' column does not exist in the data.")
    exit()

# Check if the categorical_columns and numerical_columns columns exist in the data
categorical_columns = ['categorical_column_1', 'categorical_column_2']
numerical_columns = list(set(data.columns) - set(categorical_columns))
if not set(categorical_columns).issubset(data.columns) or not set(numerical_columns).issubset(data.columns):
    print("The 'categorical_columns' or 'numerical_colum

In [ ]:
print(conversation['actual_code'])

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Load the tabular dataset
data = pd.read_csv('data.csv')

# Preprocess the data
# Handle missing values
data = data.dropna()

# Convert categorical variables to dummy variables
data = pd.get_dummies(data, columns=['categorical_column_1', 'categorical_column_2'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.drop(['target_column'], axis=1), 
                                                    data['target_column'], 
                                                    test_size=0.2, 
                                                    random_state=42)

# Train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model on the test set
score = model.score(X_test, y_test)
print('The score of the model on the test set is:', score)
```


In [ ]:
print(conversation['code_compare'])

Revised Code: 9/10
Actual Code: 7/10
